In [1]:
import numpy as np # 
import pandas as pd # 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-eb34c5ba2477>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
import json
import boto3 
import os
from io import StringIO


In [3]:
# s3 secret
ACCESS_KEY = "<KEY>" 
SECRET_KEY = "<SECRET>" 
BUCKET_NAME ='<bucket>'

In [4]:
s3 = boto3.resource(
    's3',
    region_name='us-east-1',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
).Bucket(BUCKET_NAME)

json.load_s3 = lambda f: json.load(s3.Object(key=f).get()["Body"])
json.dump_s3 = lambda obj, f: s3.Object(key=f).put(Body=json.dumps(obj))

In [20]:
# load data
fileName = 'resources/News_Category_Dataset_v2.json'

json_buffer=json.load_s3('Project3/'+fileName)
df = pd.read_json(json_buffer).T
#df = json.load_s3('Project3/'+fileName)
#df = pd.read_json(fileName, lines=True)
#json_buffer = StringIO()
#df.to_json(json_buffer, orient='index')
#json.dump_s3(json_buffer.getvalue(),'Project3/'+fileName)

df.head()

,category,headline,authors,link,short_description,date
0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.,1527292800000
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.,1527292800000
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.,1527292800000
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.,1527292800000
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment.""",1527292800000


In [21]:
df['category'].value_counts()

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL            9887 
STYLE & BEAUTY    9649 
PARENTING         8677 
HEALTHY LIVING    6694 
QUEER VOICES      6314 
FOOD & DRINK      6226 
BUSINESS          5937 
COMEDY            5175 
SPORTS            4884 
BLACK VOICES      4528 
HOME & LIVING     4195 
PARENTS           3955 
THE WORLDPOST     3664 
WEDDINGS          3651 
WOMEN             3490 
IMPACT            3459 
DIVORCE           3426 
CRIME             3405 
MEDIA             2815 
WEIRD NEWS        2670 
GREEN             2622 
WORLDPOST         2579 
RELIGION          2556 
STYLE             2254 
SCIENCE           2178 
WORLD NEWS        2177 
TASTE             2096 
TECH              2082 
MONEY             1707 
ARTS              1509 
FIFTY             1401 
GOOD NEWS         1398 
ARTS & CULTURE    1339 
ENVIRONMENT       1323 
COLLEGE           1144 
LATINO VOICES     1129 
CULTURE & ARTS    1030 
EDUCATION         1004 
Name: category, 

News API categories
* business
* entertainment
* general
* health
* science
* sports
* technology

In [22]:
business =['BUSINESS','MONEY']
entertainment=['ENTERTAINMENT','ARTS','COMEDY','ARTS & CULTURE','CULTURE & ARTS','MEDIA']
health=['HEALTH','WELLNESS','HEALTHY LIVING']
science=['SCIENCE']
sports=['SPORTS']
technology = ['TECH']

In [23]:
condition = df['category'].isin(business)
businessDF = df[condition]
businessDF.shape

(7644, 6)

In [24]:
condition = df['category'].isin(entertainment)
entertainmentDF= df[condition]
entertainmentDF.shape

(27926, 6)

In [25]:
condition = df['category'].isin(science)
scienceDF= df[condition]
scienceDF.shape

(2178, 6)

In [26]:
condition = df['category'].isin(sports)
sportsDF= df[condition]
sportsDF.shape

(4884, 6)

In [27]:
condition = df['category'].isin(technology)
technologyDF= df[condition]
technologyDF.shape

(2082, 6)

In [28]:
categories=[entertainmentDF,businessDF,scienceDF,sportsDF,technologyDF]
categoryNames=['entertainment','business','science','sports','technology']

# install Vader

 pip install vaderSentiment
 
 https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
 
 https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f
 
 


In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [30]:
analyzer = SentimentIntensityAnalyzer()
def sentimentAnalysis(art):
    sentiment_dict=analyzer.polarity_scores(art)
    if sentiment_dict['compound'] >= 0.05 : 
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05 : 
        return "Negative"
    else : 
        return "Neutral"    


ix=0
for categoryDF in categories:
    result=[]
    fileName= f'resources/{categoryNames[ix]}_labeled_data.json'
    ix+=1
    print(fileName)
    for index, row in categoryDF.iterrows():
        art= row['short_description']
        res = sentimentAnalysis(art)
        result.append(res)
    categoryDF['label']=result
    df=categoryDF[['short_description','label']]
    json_buffer = StringIO()
    df.to_json(json_buffer, orient='index')
    json.dump_s3(json_buffer.getvalue(),'Project3/'+fileName)
    print(df.shape)

# Files
* resources/entertainment_labeled_data.json
* resources/business_labeled_data.json
* resources/science_labeled_data.json
* resources/sports_labeled_data.json
* resources/technology_labeled_data.json


In [37]:
categoryNames=['entertainment','business','science','sports','technology']
for categoryname in categoryNames:
    fileName= f'resources/{categoryname}_labeled_data.json'
    json_buffer=json.load_s3('Project3/'+fileName)
    df = pd.read_json(json_buffer).T
    print(df.head())

                                                                                                               short_description  \
1  Of course it has a song.                                                                                                        
2  The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.                                         
3  The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.                                          
4  The "Dietland" actress said using the bags is a "really cathartic, therapeutic moment."                                         
5  "It is not right to equate horrific incidents of sexual assault with misplaced compliments or humor," he said in a statement.   

      label  
1  Neutral   
2  Neutral   
3  Positive  
4  Neutral   
5  Negative  
                                                                                                                           